In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
# from ROI_extraction import preprocess_image
import cv2
import os
# Set the path to dataset
dataset_path = '../images/3regimes/'

ids = []
labels = {}
classes = {'excess':1,'normal':0,'insufficient':2}
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        for filename in os.listdir(class_path) :
            if filename.endswith((".jpg", ".jpeg", ".png")):
                img_path = os.path.join(class_path, filename) 
                ids.append(img_path)
                labels[img_path]=classes[class_name]

# Set the input image dimensions
img_width, img_height = 800, 800
n_channels = 3

params = {'dim': (img_height,img_width),
          'batch_size': 64,
          'n_classes': 3,
          'n_channels': n_channels,
          'shuffle': False}

# Set the number of classes
num_classes = 3

def data_generation(list_IDs_temp):
    y = np.empty((len(list_IDs_temp)), dtype=int)
    X = []
    for i, ID in enumerate(list_IDs_temp):
        img= cv2.imread(ID)
        B_val,G_val,R_val = img[:,:,0],img[:,:,1],img[:,:,2]
        # Extract the mean values of hue, saturation, and value
        R_mean = np.mean(R_val)
        G_mean = np.mean(G_val)
        B_mean = np.mean(B_val)
        # Extract the standard deviation of hue, saturation, and value
        R_std = np.std(R_val)
        G_std = np.std(G_val)
        B_std = np.std(B_val)
        # Extract the contrast values of hue, saturation, and value
        # H_max,S_max,V_max = np.max(H_val),np.max(S_val),np.max(V_val)
        # H_min,S_min,V_min = np.min(H_val),np.min(S_val),np.min(V_val)
        # H_contrast = (H_max - H_min) / (H_max + H_min)
        # S_contrast = (S_max - S_min) / (S_max + S_min)
        # V_contrast = (V_max - V_min) / (V_max + V_min)
        X.append([R_mean,R_std,G_mean,G_std,B_mean,B_std])
        y[i] = labels[ID]
    return pd.DataFrame(X,columns=['R_mean','R_std','G_mean','G_std','B_mean','B_std']),keras.utils.to_categorical(y, num_classes=params['n_classes']),y

In [2]:
X,y_categorical,y = data_generation(ids)

In [3]:
for c in classes:
    print(f'{c}\n{X[y==classes[c]].mean()}\n')

excess
R_mean    38.651664
R_std     39.200250
G_mean    15.443444
G_std     13.228848
B_mean     8.955291
B_std      5.424075
dtype: float64

normal
R_mean    128.572889
R_std      94.871096
G_mean     60.426629
G_std      54.877638
B_mean     27.511409
B_std      17.857769
dtype: float64

insufficient
R_mean    32.739629
R_std     30.262505
G_mean    13.375520
G_std     10.072094
B_mean     8.326216
B_std      4.567133
dtype: float64

